In [1]:
import numpy as np
import nltk
from collections import Counter

In [2]:
corpus = nltk.corpus.brown.tagged_sents(tagset='universal')[:10000]
print(len(corpus))
print(type(corpus))
print(corpus[:10])

10000
<class 'nltk.collections.LazySubsequence'>
[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks

In [3]:
flat_corpus = [item for sublist in corpus for item in sublist]
print(len(flat_corpus))
print(flat_corpus[:10])

219770
[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP')]


## Question 1: generate the transition matrix, observation matrix, and initial state distribution

### Transition matrix

In [4]:
tags = [[x[1] for x in y] for y in corpus]

In [5]:
def make_ngram(token, n, pred=False):
    if len(token) == 0 and pred:
        ngrams = ['' for _ in range(n - 1)]
        return ngrams
    ngrams = zip(*[token[i:] for i in range(n)])
    ngrams = list(ngrams)
    if pred:
        ngrams = [x[1:] for x in ngrams]
        return ngrams
    return ngrams

In [6]:
tags_bigram = []
for tag in tags:
    tags_bigram.append(make_ngram(tag, 2))

In [7]:
tags_bigram_flat = [item for sublist in tags_bigram for item in sublist]

In [8]:
tags_freq = Counter(tags_bigram_flat)
tags_freq = tags_freq.most_common(len(tags_freq))
tags_freq[:10]

[(('DET', 'NOUN'), 15796),
 (('NOUN', '.'), 15743),
 (('NOUN', 'ADP'), 13604),
 (('NOUN', 'NOUN'), 12624),
 (('ADP', 'DET'), 12035),
 (('ADJ', 'NOUN'), 11071),
 (('NOUN', 'VERB'), 8568),
 (('ADP', 'NOUN'), 7802),
 (('DET', 'ADJ'), 6424),
 (('VERB', 'VERB'), 6266)]

In [9]:
unique_tags = np.unique([item for sublist in tags for item in sublist])
ind_tag_map = {k: v for k, v in enumerate(unique_tags)}
transition_matrix = np.zeros(shape=(12, 12))

for idx, tag in enumerate(unique_tags):
    placeholder = sorted(list(filter(lambda x: x[0][0] == tag, tags_freq)),
                         key=lambda x: x[0][1])
    placeholder = [x[1] for x in placeholder]
    try:
        transition_matrix[:, idx] = placeholder
    except ValueError:
        continue

x = sorted(list(filter(lambda x: x[0][0] == "X", tags_freq)), key=lambda x: x[0][1])
x = [x[1] for x in x]
x.insert(8, 0)
pron = sorted(list(filter(lambda x: x[0][0] == "PRON", tags_freq)), key=lambda x: x[0][1])
pron = [x[1] for x in pron]
pron.append(0)
transition_matrix[:, 8] = pron
transition_matrix[:, 11] = x
transition_matrix = transition_matrix + 1  # smoothing
norm = np.sum(transition_matrix, axis=1)
transition_matrix = transition_matrix / norm[:, None]

In [10]:
transition_matrix

array([[9.29421340e-02, 5.68970195e-02, 1.20150382e-02, 5.11995659e-02,
        5.93000271e-03, 1.28677183e-02, 6.10208907e-01, 3.43009961e-02,
        2.19758924e-02, 9.18569048e-03, 8.98414790e-02, 2.63555676e-03],
       [5.58851077e-02, 6.07555417e-02, 1.43490478e-01, 8.86668748e-02,
        4.86418982e-02, 4.01186388e-01, 5.82578832e-02, 1.49235092e-02,
        3.99625351e-03, 6.05682173e-03, 1.18014362e-01, 1.24882922e-04],
       [7.26390387e-02, 5.04159606e-02, 1.98351564e-02, 5.03004160e-02,
        1.72931752e-02, 8.93544908e-03, 5.23994762e-01, 1.88337698e-02,
        1.32491142e-02, 1.74087198e-02, 2.06362656e-01, 7.31782468e-04],
       [1.22984800e-01, 1.45094427e-02, 4.27222478e-02, 1.04790419e-01,
        6.72501152e-02, 5.58498388e-02, 1.56379549e-01, 1.28972824e-02,
        4.73284201e-02, 1.79640719e-02, 3.56863197e-01, 4.60617227e-04],
       [2.61305706e-01, 8.99523105e-02, 5.92007893e-03, 2.13780628e-02,
        6.57786548e-04, 2.30225292e-03, 5.13895741e-01, 1.94

### Observation matrix

In [44]:
obs_freq = Counter(flat_corpus)
obs_freq = obs_freq.most_common(len(obs_freq))
obs_freq[:5]

[(('the', 'DET'), 12054),
 ((',', '.'), 11133),
 (('.', '.'), 8560),
 (('of', 'ADP'), 6770),
 (('and', 'CONJ'), 4935)]

In [79]:
unique_words = np.unique([x[0][0] for x in obs_freq])
ind_word_map = {k: v for k, v in enumerate(unique_words)}

obs_matrix = np.zeros(shape=(len(unique_words), len(unique_tags)))

In [88]:
for row, word in enumerate(unique_words):
    for col, tag in enumerate(unique_tags):
        filtered_col = list(filter(lambda x: x[0][1] == tag, obs_freq))
        filtered_row = list(filter(lambda x: x[0][0] == word, filtered_col))
        if len(filtered_row) > 0:
            obs_matrix[row, col] = filtered_row[0][1]

In [61]:
#np.save("obs_matrix.pkl", obs_matrix, allow_pickle=True)
obs_matrix = np.load("obs_matrix.pkl.npy")
obs_matrix = obs_matrix + 1

In [62]:
print(obs_matrix.shape)
unk = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]).astype(float)
obs_matrix = np.vstack([obs_matrix, unk])

(23488, 12)


In [63]:
norm_obs = np.sum(obs_matrix, axis=0)
obs_matrix_norm = obs_matrix / norm_obs[None, :]

In [16]:
#np.save("obs_matrix_norm", obs_matrix_norm, allow_pickle=True)

## Initial state distribution

In [64]:
first_words = [x[0] for x in corpus]

In [65]:
first_words = [x[1] for x in first_words]

In [66]:
first_words_freq = Counter(first_words)
first_words_freq = first_words_freq.most_common(len(first_words_freq))

In [67]:
first_words_freq = sorted(first_words_freq, key=lambda x: x[0])
initial_state_dist = [x[1] / sum(y[1] for y in first_words_freq) for x in first_words_freq]

### Viterbi algorithm

In [68]:
obs_matrix_norm = obs_matrix_norm.T
obs_matrix_norm.shape

(12, 23489)

In [122]:
def viterbi(obs, pi, A, B):
    pi_log = np.log(pi)
    A_log = np.log(A)
    B_log = np.log(B)
    states = A.shape[0]
    n = len(obs)
    print(n)

    D_log = np.zeros((states, n))
    E = np.zeros((states, n - 1)).astype(int)
    D_log[:, 0] = pi_log + B_log[:, obs[0]]

    for j in range(1, n):
        for i in range(states):
            temp_sum = A_log[:, i] + D_log[:, j - 1]
            D_log[i, j] = np.max(temp_sum) + B_log[i, obs[j]]
            E[i, j - 1] = np.argmax(temp_sum)
    print(n)
    S_opt = np.zeros(n).astype(int)
    print(len(S_opt))
    S_opt[-1] = np.argmax(D_log[:, -1])
    for n in range(n - 2, -1, -1):
        S_opt[n] = E[int(S_opt[n + 1]), n]
    print(len(S_opt))
    return S_opt

In [114]:
for_test = nltk.corpus.brown.tagged_sents(tagset="universal")[10150:10153]
for_test

[[('Those', 'DET'),
  ('coming', 'VERB'),
  ('from', 'ADP'),
  ('other', 'ADJ'),
  ('denominations', 'NOUN'),
  ('will', 'VERB'),
  ('welcome', 'VERB'),
  ('the', 'DET'),
  ('opportunity', 'NOUN'),
  ('to', 'PRT'),
  ('become', 'VERB'),
  ('informed', 'VERB'),
  ('.', '.')],
 [('The', 'DET'),
  ('preparatory', 'ADJ'),
  ('class', 'NOUN'),
  ('is', 'VERB'),
  ('an', 'DET'),
  ('introductory', 'ADJ'),
  ('face-to-face', 'ADJ'),
  ('group', 'NOUN'),
  ('in', 'ADP'),
  ('which', 'DET'),
  ('new', 'ADJ'),
  ('members', 'NOUN'),
  ('become', 'VERB'),
  ('acquainted', 'VERB'),
  ('with', 'ADP'),
  ('one', 'NUM'),
  ('another', 'DET'),
  ('.', '.')],
 [('It', 'PRON'),
  ('provides', 'VERB'),
  ('a', 'DET'),
  ('natural', 'ADJ'),
  ('transition', 'NOUN'),
  ('into', 'ADP'),
  ('the', 'DET'),
  ('life', 'NOUN'),
  ('of', 'ADP'),
  ('the', 'DET'),
  ('local', 'ADJ'),
  ('church', 'NOUN'),
  ('and', 'CONJ'),
  ('its', 'DET'),
  ('organizations', 'NOUN'),
  ('.', '.')]]

In [115]:
unique_words = np.unique([x[0][0] for x in obs_freq])
ind_word_map = {k: v for k, v in enumerate(unique_words)}
word_ind_map = {k: v for v, k in ind_word_map.items()}

In [135]:
test1 = [x[0] for x in for_test[2]]
answer1 = [x[1] for x in for_test[2]]
obs1 = [word_ind_map.get(word, 23488) for word in test1]
print(len(obs1))

16


In [136]:
a = viterbi(obs1, initial_state_dist, transition_matrix, obs_matrix_norm)
a1 = [ind_tag_map[x] for x in a]

16
16
16
16


In [137]:
a1

['PRON',
 'VERB',
 'DET',
 'ADP',
 'NOUN',
 'ADP',
 'DET',
 'NOUN',
 'ADP',
 'DET',
 'ADP',
 'NOUN',
 'CONJ',
 '.',
 'NOUN',
 '.']

In [138]:
sum([x == y for x, y in zip(answer1, a1)]) / len(answer1)

0.8125

In [139]:
print(a1)

['PRON', 'VERB', 'DET', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'ADP', 'NOUN', 'CONJ', '.', 'NOUN', '.']


In [140]:
print(answer1)

['PRON', 'VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'CONJ', 'DET', 'NOUN', '.']


In [142]:
sum([x == y for x, y in zip(answer1, a1)])

13

In [146]:
type(a.tolist())

list

In [147]:
a.tolist()

[8, 10, 5, 2, 6, 2, 5, 6, 2, 5, 2, 6, 4, 0, 6, 0]